In [1]:

#import libraries

import numpy as np
import os
import cv2
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt


#loading datasets and labels

dire = "D:/dataset trail"
categories = ['defects', 'normal']

images = []
labels = []

SIZE = 200

for category in categories:
    path = os.path.join(dire, category)
    label = categories.index(category)
    
    for img in os.listdir(path):
        imgpath = os.path.join(path, img)
        tree_img = cv2.imread(imgpath, 0)
        try:
            tree_img = cv2.resize(tree_img, (SIZE, SIZE))
            images.append(tree_img)
            labels.append(label)
        except Exception as e:
            pass

images = np.array(images)
labels = np.array(labels)


#feature extraction using GLCM (Use other better feature extractors for good results)

def feature_extractor(dataset):
    image_dataset = pd.DataFrame()
    for img in dataset:
        glcm = cv2.calcHist([img.astype(np.uint8)], [0], None, [256], [0, 256])
        contrast = np.sum((np.arange(glcm.shape[0]) - np.arange(glcm.shape[1])) ** 2 * glcm)
        dissimilarity = np.sum(np.abs(np.arange(glcm.shape[0]) - np.arange(glcm.shape[1])) * glcm)
        homogeneity = np.sum(glcm / (1 + np.abs(np.arange(glcm.shape[0]) - np.arange(glcm.shape[1]))))
        energy = np.sum(glcm ** 2)
        
        df = pd.DataFrame({
            'Energy': energy,
            'Diss_sim': dissimilarity,
            'Homogen': homogeneity,
            'Contrast': contrast
        }, index=[0])
        
        image_dataset = pd.concat([image_dataset, df], ignore_index=True)
        
    return image_dataset


X_for_ML = feature_extractor(images)


# Train-validation-test split (60-20-20)
x_train_val, x_test, y_train_val, y_test = train_test_split(X_for_ML, labels, test_size=0.2, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=0.25, random_state=42)

# Feature scaling
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_val_scaled = scaler.transform(x_val)
x_test_scaled = scaler.transform(x_test)


#Random forest declartion
def fitness_function(feature_mask):
    X_train_subset = x_train_scaled[:, feature_mask]
    X_val_subset = x_val_scaled[:, feature_mask]
    
    clf = RandomForestClassifier()
    clf.fit(X_train_subset, y_train)
    return clf.score(X_val_subset, y_val)


#Aco 
class ACO:
    def __init__(self, num_ants, num_features, alpha, beta, evaporation_rate, q0):
        self.num_ants = num_ants
        self.num_features = num_features
        self.alpha = alpha
        self.beta = beta
        self.evaporation_rate = evaporation_rate
        self.q0 = q0
        self.pheromone_matrix = np.ones((num_features, 2)) / num_features

    def run(self, num_iterations):
        best_feature_mask = None
        best_fitness = 0
        for iteration in range(num_iterations):
            ant_fitnesses = []
            for ant in range(self.num_ants):
                feature_mask = self.select_features()
                ant_fitness = fitness_function(feature_mask)
                ant_fitnesses.append((feature_mask, ant_fitness))
                if ant_fitness > best_fitness:
                    best_fitness = ant_fitness
                    best_feature_mask = feature_mask
            self.update_pheromone(ant_fitnesses)
        return best_feature_mask

    def select_features(self):
        feature_mask = np.zeros(self.num_features, dtype=bool)
        while not feature_mask.any():
            q = np.random.uniform()
            if q <= self.q0:
                feature_index = self.select_best_feature()
            else:
                feature_index = self.select_random_feature()
            feature_mask[feature_index] = True
        return feature_mask

    def select_best_feature(self):
        heuristic_info = self.pheromone_matrix[:, 0] ** self.alpha * (1 / self.pheromone_matrix[:, 1]) ** self.beta
        return np.argmax(heuristic_info)

    def select_random_feature(self):
        return np.random.randint(self.num_features)

    def update_pheromone(self, ant_fitnesses):
        self.pheromone_matrix *= self.evaporation_rate
        for feature_index in range(self.num_features):
            for ant_fitness in ant_fitnesses:
                if ant_fitness[0][feature_index]:
                    self.pheromone_matrix[feature_index, 0] += 1 / ant_fitness[1]

# Parameters for the ACO algorithm
num_ants = 20
num_features = x_train_scaled.shape[1]
alpha = 1
beta = 1
evaporation_rate = 0.3
q0 = 0.9
num_iterations = 100

# Run ACO algorithm
aco = ACO(num_ants, num_features, alpha, beta, evaporation_rate, q0)
best_feature_mask = aco.run(num_iterations)

# Evaluate performance of selected features on validation set
X_val_selected = x_val_scaled[:, best_feature_mask]
clf = RandomForestClassifier()
clf.fit(X_val_selected, y_val)
y_val_pred = clf.predict(X_val_selected)
val_accuracy = accuracy_score(y_val, y_val_pred)
val_classification_report = classification_report(y_val, y_val_pred, target_names=categories)

# Train SVM model on selected features using training + validation set
X_train_val_selected = np.concatenate((x_train_scaled[:, best_feature_mask], X_val_selected), axis=0)
y_train_val_combined = np.concatenate((y_train, y_val), axis=0)
clf.fit(X_train_val_selected, y_train_val_combined)

# Evaluate performance on testing set
X_test_selected = x_test_scaled[:, best_feature_mask]
y_test_pred = clf.predict(X_test_selected)
test_accuracy = accuracy_score(y_test, y_test_pred)
test_classification_report = classification_report(y_test, y_test_pred, target_names=categories)

# Print accuracies and classification reports
print("Validation Accuracy:", val_accuracy)
print("Validation Classification Report:")
print(val_classification_report)

print("Test Accuracy:", test_accuracy)
print("Test Classification Report:")
print(test_classification_report)


Validation Accuracy: 0.984375
Validation Classification Report:
              precision    recall  f1-score   support

     defects       1.00      0.97      0.98        29
      normal       0.97      1.00      0.99        35

    accuracy                           0.98        64
   macro avg       0.99      0.98      0.98        64
weighted avg       0.98      0.98      0.98        64

Test Accuracy: 0.671875
Test Classification Report:
              precision    recall  f1-score   support

     defects       0.62      0.72      0.67        29
      normal       0.73      0.63      0.68        35

    accuracy                           0.67        64
   macro avg       0.68      0.68      0.67        64
weighted avg       0.68      0.67      0.67        64

